In [23]:
from datasets import DatasetDict, Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

import evaluate
import numpy as np
from transformers import DataCollatorWithPadding

import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
#TODO
# Fix the tensors at the bottom
# Finish fine tuning

In [7]:
dataset_dict = load_dataset("csv", data_files="data/gutenberg/uniform_excerpts.csv")

Generating train split: 0 examples [00:00, ? examples/s]

In [24]:
df = pd.read_csv("data/gutenberg/uniform_excerpts.csv")

In [27]:
x = df['text']
y = df['label']

In [38]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=104,test_size=0.15, shuffle=True)

In [43]:
train_dataset = Dataset.from_pandas(pd.DataFrame({"text": x_train, "label": y_train}))
eval_dataset = Dataset.from_pandas(pd.DataFrame({"text": x_test, "label": y_test}))

In [8]:
#define pre-trained model path
model_path = "google-bert/bert-base-uncased"

#load model tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [10]:
#load model with binary classifcation heaed
id2label = {0: "1400", 1: "1500", 2: "1600", 3: "1700", 4: "1800", 5: "1900"}
label2id = {1400: "0", 1500: "1", 1600: "2", 1700: "3", 1800: "4", 1900: "5"}
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels = 6, id2label=id2label, label2id=label2id,)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
#freeze all base model parameters
for name, param in model.base_model.named_parameters():
    param.requires_grad = False

#unfreeze base model pooling layers
for name, param in model.base_model.named_parameters():
    if "pooler" in name:
        param.requires_grad = True

In [49]:
#define text preprocessing
def preprocess_function(examples):
    #return tokenized text with truncation and padding
    return tokenizer(examples["text"], truncation=True, padding=True)

#preprocess all datasets
train_dataset = train_dataset.map(preprocess_function, batched=True)
eval_dataset = eval_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/5100 [00:00<?, ? examples/s]

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

In [13]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [14]:
#load metrics
accuracy = evaluate.load("accuracy")
auc_score = evaluate.load("roc_auc")

def compute_metrics(eval_pred):
    #get predictions
    predictions, labels = eval_pred

    #softmax to get probabilities
    probabilities = np.exp(predictions)/ np.exp(predictions).sum(-1, keepdims=True)
    positive_class_probs = probabilities[:, 1]

    #compute auc
    auc = np.round(auc_score.compute(prediction_scores=positive_class_probs, references=labels)["roc_auc"], 3)

    #predict most probable class
    predicted_classes = np.argmax(predictions, axis=1)

    acc = np.round(accuracy.compute(predictions=predicted_classes, references=labels)["accuracy"], 3)

    return {"Accuracy": acc, "AUC": auc}

In [21]:
#hyperparameters
lr = 2e-4
batch_size = 8
num_epochs = 10

training_args = TrainingArguments(
    output_dir = "bert-english-classifier_teacher",
    learning_rate = lr,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    num_train_epochs = num_epochs,
    logging_strategy = "epoch",
    eval_strategy = "epoch",
    save_strategy = "epoch",
    load_best_model_at_end = True,
)

In [50]:
#TODO: When running train_dataset and eval_dataset, there is a tensor error where the trainer cant create a tensor. From what I gather, this might be
# issue pertaining to the label in the uniform_excerpts dataset which isn't a flat integer.
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

C:\Users\james\AppData\Local\Temp\ipykernel_2556\438451520.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`label` in this case) have excessive nesting (inputs type `list` where type `int` is expected).